In [1]:
# Importing dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
from config import weather_api_key
import requests
import time
from datetime import datetime

In [3]:
# Creating an array of sets of random Latitudes and Longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000 )

# Pairing the latitudes and longitudes
latLngs = zip(lats,lngs)
latLngs

In [4]:
# Adding the latitudes & longitudes to a list
coordinates = list(latLngs)

In [5]:
# Creating list to hold cities
cities = []

# Looping through coordinates finding the nearest city
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # Checking to see if the city is unique. If so, it is added to the "cities" list.
    if city not in cities:
        cities.append(city)

# Checking how many cities we have
len(cities)

747

In [8]:
# Creating the base URL
url = 'https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + weather_api_key

# Creating an empty list to hold the weather data
cityData = []

# Printing the beginning  of the logging
print('Beginning Data Retrieval      ')
print('------------------------------')

# Creating counters
recordCount = 1
setCount = 1

# Looping through cities in 'cities' list
for i, city in enumerate(cities):
    # Grouping cities into sets of 50 for logging purposes
    if i % 50 == 0 and i >= 50:
        setCount += 1
        recordCount = 1
        time.sleep(60)

    # Creating endpoint URL with each city
    cityURL = url + '&q=' + city.replace(' ','+')

    # Logging the URL, Record and set numbers and the city.
    print(f'Processing Record {recordCount} of Set {setCount} | {city}')

    #Adding 1 to record count
    recordCount += 1

    # Making API request for each city
    try:
        # Parsing the JSON and retrieving the data
        cityWeather = requests.get(cityURL).json()

        # Parsing out the needed data
        cityLat = cityWeather['coord']['lat']
        cityLng = cityWeather['coord']['lon']
        cityMaxTemp = cityWeather['main']['temp_max']
        cityHumidity = cityWeather['main']['humidity']
        cityClouds = cityWeather['clouds']['all']
        cityWind = cityWeather['wind']['speed']
        cityCountry = cityWeather['sys']['country']
        cityDescription = cityWeather['weather'][0]['description']

        # Appending the city info into the cityData List
        cityData.append({'City': city.title(),
                         'Country': cityCountry,
                         'Lat':cityLat,
                         'Lng':cityLng,
                         'Max Temp':cityMaxTemp,
                         'Humidity':cityHumidity,
                         'Cloudiness':cityClouds,
                         'Wind Speed':cityWind,
                         'Current Description':cityDescription
                         })

    # If an error is experienced, skip the city.
    except:
        print('City not found. Skipping City...')
        pass

#Indicate that data loading is complete
print('--------------------------')
print('Data Retrieval Complete!')
print('--------------------------')

Beginning Data Retrieval      
------------------------------
Processing Record 1 of Set 1 | taolanaro
City not found. Skipping City...
Processing Record 2 of Set 1 | saint-philippe
Processing Record 3 of Set 1 | tutoia
Processing Record 4 of Set 1 | severo-kurilsk
Processing Record 5 of Set 1 | hammerfest
Processing Record 6 of Set 1 | bereda
Processing Record 7 of Set 1 | provideniya
Processing Record 8 of Set 1 | kavieng
Processing Record 9 of Set 1 | jamestown
Processing Record 10 of Set 1 | busselton
Processing Record 11 of Set 1 | zinder
Processing Record 12 of Set 1 | souillac
Processing Record 13 of Set 1 | cape town
Processing Record 14 of Set 1 | saldanha
Processing Record 15 of Set 1 | yanam
Processing Record 16 of Set 1 | richards bay
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | pisco
Processing Record 19 of Set 1 | cabo san lucas
Processing Record 20 of Set 1 | dikson
Processing Record 21 of Set 1 | teodoro sampaio
Processing Record 22 of Set 1 |

In [9]:
# Checking the amount of cities we have
len(cityData)

688

In [11]:
# Converting the array of dictionaries to a DataFrame
cityDataDf = pd.DataFrame(cityData)

cityDataDf.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Saint-Philippe,RE,-21.3585,55.7679,76.59,84,29,10.13,scattered clouds
1,Tutoia,BR,-2.7619,-42.2744,80.24,84,73,10.58,light rain
2,Severo-Kurilsk,RU,50.6789,156.1250,32.50,62,68,22.59,broken clouds
3,Hammerfest,NO,70.6634,23.6821,18.05,67,44,5.75,scattered clouds
4,Bereda,ES,43.2686,-7.5406,47.95,100,82,5.77,broken clouds


In [14]:
# Creating the output file
outputDataFile = 'WeatherPy_Database.csv'

# Exporting the cityData into a CSV
cityDataDf.to_csv(outputDataFile, index_label='City_ID')